In [21]:
import pandas as pd
import numpy as np

# load data
blocks = pd.read_csv('data_census//tl_2010_06075_tabblock10.csv')
caseid_block = pd.read_csv('caseid_block_all.csv')
cases = pd.read_csv('311_cases_clean.csv')

In [22]:
# create the date range
#'2016-05-01'
date_range_use = pd.date_range('2018-04-01','2018-05-01', freq = '6H')

# create the columns
date_col = list(date_range_use)*len(blocks['GEOID10'].unique())
block_col = sorted(list(blocks['GEOID10'].unique())*len(date_range_use))

# create the empty matrix
empty_matrix = pd.DataFrame({'Opened_rnd':date_col,
                          'block_fips':block_col})

In [23]:
# get the hour
cases['hour'] = cases['Opened'].apply(lambda x:int(x[11:13]))

def bin_hours(row):
    """
    will group hours together in order to have bins instead of the raw timestamp
    """
    if row['hour'] < 6:
        return '0' 
    elif row['hour'] < 12:
        return '6' 
    elif row['hour'] < 18:
        return '12'
    else:
        return '18'

# insert the rounded hour back into the dataframe
cases['bin_hour'] = cases.apply(bin_hours, axis=1)
cases['new_date'] = cases['Opened_rnd'].apply(lambda x:x[0:10])
cases['Opened_rnd'] = cases['new_date'] + ' '+ cases['bin_hour'] + ':00:00'

In [27]:
# select appropriate columns
caseid_block = caseid_block.loc[:,[
    
    'CaseID',
    'block_fips',
]]

cases = cases.loc[:,[
    
    'CaseID',
    'Opened_rnd',
]]

# join tables together
case_block_date = cases.merge(caseid_block, on = 'CaseID')
model_data = empty_matrix.merge(case_block_date,
                                on = ['block_fips','Opened_rnd'],
                                how = 'left')

# create target feature
model_data['poop'] = np.where(model_data['CaseID'].notnull(), 1, 0)
model_data = model_data.reset_index(drop = True)

In [28]:
# save to hdf
model_data.to_hdf('data_model//model_data_test.h5', key = 'xyz', complib = 'blosc')

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->datetime,key->block2_values] [items->['Opened_rnd']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
